<a href="https://colab.research.google.com/github/Jeorr/ML/blob/main/Lab2Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import zipfile
import shutil
import requests
from tensorflow.keras import layers, models, utils
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Скачуємо архів
url = 'https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip';
dest_folder = '/content/drive/MyDrive/ML/';
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
filename = url.split('/')[-1];
file_path = os.path.join(dest_folder, filename)

with requests.get(url, stream=True) as r:
    with open(file_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

# Розархівовуємо zip файл, якщо він існує
zip_path = '/content/drive/MyDrive/ML/cats_and_dogs.zip'
extract_to_path = '/content/Lab2'
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Файли успішно розархівовано в {extract_to_path}")
else:
    print(f"Файл {zip_path} не знайдено.")

# Підготовка шляхів до наборів даних
train_dir = '/content/Lab2/cats_and_dogs/train'
validation_dir = '/content/Lab2/cats_and_dogs/validation'
test_dir = '/content/Lab2/cats_and_dogs/test'

# Генератор для тренувальних даних
train_generator = utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

# Генератор для валідаційних даних
validation_generator = utils.image_dataset_from_directory(
    validation_dir,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

# спроба збудувати модель з нуля провалилася, після багатьох спроб вона так і не змогла демонструвати
# хороші результати. Спробуємо попередньо навчену модель як базову для нашої
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True  # Розморозити всі шари

# Заморозити лише верхні шари для збереження попереднього навчання
for layer in base_model.layers[:100]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])

# Компіляція з меншою швидкістю навчання
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
)

# Передбачення
prediction_based_on_validation_data = model.predict(validation_generator)
# Округлення та перетворення на цілі числа (0 або 1)
prediction_based_on_validation_data = np.round(prediction_based_on_validation_data).flatten()

# Справжні мітки
real_data = np.concatenate([y for x, y in validation_generator], axis=0)

# Обчислення MAE та MSE
mae = mean_absolute_error(real_data, prediction_based_on_validation_data)
mse = mean_squared_error(real_data, prediction_based_on_validation_data)

# Виведення результатів
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")

# оскільки тестови картинки в нас не розбиті по категоріям, перевірити відповідність
# тестовим даним нам доведеться робити вручну
# для генератора тестових даних відключаємо мітки, відключаємо перемішування
test_generator = utils.image_dataset_from_directory(
    test_dir,
    labels=None,
    label_mode=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(150, 150),
    shuffle=False,
    verbose=True,
)

# Передбачення на тестових даних
prediction_based_on_test_data = model.predict(test_generator)

# Інтерпретація результатів
predicted_classes = np.round(prediction_based_on_test_data).flatten()

# Отримання назв файлів
file_names = test_generator.file_paths

# Створюємо DataFrame для результатів
results = pd.DataFrame({
    "FileName": file_names,
    "PredictedClass": ["Dog" if cls == 1 else "Cat" for cls in predicted_classes.flatten()]
})

# Виводимо результати в консоль
print(results)

Файли успішно розархівовано в /content/Lab2
Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 258s 4s/step - accuracy: 0.8809 - loss: 0.2723 - val_accuracy: 0.9630 - val_loss: 0.2774
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 0.9668 - loss: 0.0896 - val_accuracy: 0.9680 - val_loss: 0.1285
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.9829 - loss: 0.0413 - val_accuracy: 0.9580 - val_loss: 0.2163
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 216s 3s/step - accuracy: 0.9828 - loss: 0.0495 - val_accuracy: 0.9730 - val_loss: 0.0967
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - accuracy: 0.9854 - loss: 0.0394 - val_accuracy: 0.9390 - val_loss: 0.1806
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.9781 - loss: 0.0601 - val_accuracy: 0.9710 - val_loss: 0.0980
Epoch 7/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 165s 3s/step - accuracy